In [1]:
# !pip install inference

In [2]:
# # install the Roboflow Inference SDK if not installed
# # pip install inference

# from inference import get_model
# import supervision as sv
# import cv2

# # Load your own Roboflow model
# # api_key = 'MsTpUtZbxPcYwekvmhUd'
# # project_id = 'muzzle-detection1'
# model = get_model(
#     model_id="nitesh-gujjar/muzzle-detection1/4",  # <-- replace this
#     api_key="MsTpUtZbxPcYwekvmhUd"                      # <-- required if model is private
# )

# # Read the image
# image = cv2.imread("your-image.jpg")

# # Run inference
# results = model.infer(image)[0]

# # Convert to Supervision Detections
# detections = sv.Detections.from_inference(results)

# # Create annotators
# bounding_box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# # Annotate the image
# annotated_image = bounding_box_annotator.annotate(scene=image, detections=detections)
# annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

# # Display
# sv.plot_image(annotated_image)


In [ ]:
import os
from dotenv import load_dotenv
import roboflow
import cv2

def crop_muzzle_images(input_folder: str, output_folder: str):
    # Load environment variables from .env file
    load_dotenv()

    # Read api_key and model_id
    api_key = os.getenv("api_key")
    model_id = os.getenv("project_id")  # e.g., "your-project-name"

    # Initialize Roboflow
    rf = roboflow.Roboflow(api_key=api_key)
    workspace = rf.workspace()

    # Load the project and model
    project = workspace.project(model_id)
    model = project.version(3).model  # change and try version 4 if some images were left version

    # Set confidence and overlap thresholds
    model.confidence = 30
    model.overlap = 50

    # Supported image extensions
    supported_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Loop over all images in the input folder
    for filename in os.listdir(input_folder):
        if any(filename.lower().endswith(ext) for ext in supported_extensions):
            image_path = os.path.join(input_folder, filename)

            try:
                # Read the image
                image = cv2.imread(image_path)

                # Run prediction
                prediction = model.predict(image_path)

                # Convert predictions to JSON
                predictions_json = prediction.json()

                # Check if there are any detections
                if len(predictions_json['predictions']) == 0:
                    print(f"No muzzle detected in {filename}. Skipping...")
                    continue

                # Find the prediction with highest confidence
                best_pred = max(predictions_json['predictions'], key=lambda x: x['confidence'])

                # Get bounding box coordinates
                x = int(best_pred['x'])
                y = int(best_pred['y'])
                width = int(best_pred['width'])
                height = int(best_pred['height'])

                # Calculate top-left and bottom-right corners
                x1 = max(x - width // 2, 0)
                y1 = max(y - height // 2, 0)
                x2 = min(x + width // 2, image.shape[1])
                y2 = min(y + height // 2, image.shape[0])

                # Crop the image
                cropped_image = image[y1:y2, x1:x2]

                # Build output image path
                output_path = os.path.join(output_folder, filename)

                # Save the cropped image
                cv2.imwrite(output_path, cropped_image)

                print(f"Cropped and saved: {output_path}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    print("✅ Cropping completed for all images.")

# Example usage:
if __name__ == "__main__":
    input_folder = "../60_images_of_6_cows/db-images2/207"
    output_folder = "../60_images_of_6_cows/db-images2/cropped_img207/"
    crop_muzzle_images(input_folder, output_folder)


loading Roboflow workspace...
loading Roboflow project...
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_10.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_9.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_8.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_5.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_4.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_6.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_7.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_3.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_2.jpg
Cropped and saved: ../60_images_of_6_cows/db-images2/cropped_img207/7_1.jpg
✅ Cropping completed for all images.
